# B2 Conditioning Gaussian processes

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import scipy

In [ ]:
#running Seb's code

%run ../../taskb.ipynb

In [66]:
# generate data x

l=101
x=torch.tensor(np.linspace(0,1,l))
print(x)

# generate weights w

w=torch.ones_like(x)*1/(l-1)
w[0]=0.5/(l-1)
w[-1]=0.5/(l-1)
print(w)

#note that the data is row vectors in the code

tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900, 0.1000, 0.1100, 0.1200, 0.1300, 0.1400, 0.1500, 0.1600, 0.1700,
        0.1800, 0.1900, 0.2000, 0.2100, 0.2200, 0.2300, 0.2400, 0.2500, 0.2600,
        0.2700, 0.2800, 0.2900, 0.3000, 0.3100, 0.3200, 0.3300, 0.3400, 0.3500,
        0.3600, 0.3700, 0.3800, 0.3900, 0.4000, 0.4100, 0.4200, 0.4300, 0.4400,
        0.4500, 0.4600, 0.4700, 0.4800, 0.4900, 0.5000, 0.5100, 0.5200, 0.5300,
        0.5400, 0.5500, 0.5600, 0.5700, 0.5800, 0.5900, 0.6000, 0.6100, 0.6200,
        0.6300, 0.6400, 0.6500, 0.6600, 0.6700, 0.6800, 0.6900, 0.7000, 0.7100,
        0.7200, 0.7300, 0.7400, 0.7500, 0.7600, 0.7700, 0.7800, 0.7900, 0.8000,
        0.8100, 0.8200, 0.8300, 0.8400, 0.8500, 0.8600, 0.8700, 0.8800, 0.8900,
        0.9000, 0.9100, 0.9200, 0.9300, 0.9400, 0.9500, 0.9600, 0.9700, 0.9800,
        0.9900, 1.0000], dtype=torch.float64)
tensor([0.0050, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.

In [67]:
def inner_product(x,w,function):
    return torch.sum(function(x)*w)

In [82]:
def identity(x: float) -> float:
    return x

#test inner product
print(inner_product(x,w,identity))


tensor(0.5000, dtype=torch.float64)


In [121]:
# define a kernel function

#TODO: change to actual kernel function later

def gaussian_kernel(X,Xprime, gamma=1):
    x=X.reshape(-1,1)
    xprime=Xprime.reshape(-1,1)
    dists = scipy.spatial.distance.cdist(x,xprime,metric='sqeuclidean')
    dists = torch.tensor(dists)  # Convert numpy array to PyTorch tensor
    return torch.exp(-gamma*dists)

print(gaussian_kernel(x,x[1]))
print(gaussian_kernel(x,x)[1,:])

tensor([[0.9999],
        [1.0000],
        [0.9999],
        [0.9996],
        [0.9991],
        [0.9984],
        [0.9975],
        [0.9964],
        [0.9951],
        [0.9936],
        [0.9919],
        [0.9900],
        [0.9880],
        [0.9857],
        [0.9832],
        [0.9806],
        [0.9778],
        [0.9747],
        [0.9715],
        [0.9681],
        [0.9645],
        [0.9608],
        [0.9569],
        [0.9528],
        [0.9485],
        [0.9440],
        [0.9394],
        [0.9346],
        [0.9297],
        [0.9246],
        [0.9193],
        [0.9139],
        [0.9084],
        [0.9027],
        [0.8968],
        [0.8908],
        [0.8847],
        [0.8784],
        [0.8721],
        [0.8655],
        [0.8589],
        [0.8521],
        [0.8453],
        [0.8383],
        [0.8312],
        [0.8240],
        [0.8167],
        [0.8093],
        [0.8018],
        [0.7942],
        [0.7865],
        [0.7788],
        [0.7710],
        [0.7631],
        [0.7551],
        [0

Derive the probability distribution of the random variable $(\hat{q},f)|X$.

The random variable is normally distributed with mean $0$ and variance $Qk(X)Q^T$, where $k(X)=(k(x_i,x_j))_{i,j=1}^n$ and $Q=(w, 1_l)^T\in \mathbb{R}^{(l+1)\times(l+1)}$ a matrix with weights $(w_i)_{i=1}^n$ stacked on top of the identity matrix in $l$ dimensions $1_l$.



Note: $w\in \mathbb{R}^{l\times1}$ 

Similarly, the distribution of the random variable $f|X,\hat{q}$ can be calculated to be the following:

The random variable is normally distributed with mean $\mu_{2|1}=(\langle w,k(x,x_i)\rangle)_{i=1}^n\cdot \frac{\hat{q}}{w^Tk(X)w}$ and covariance $\Sigma_{2|1}=k(X)-(\langle w,k(x,x_i)\rangle)_{i=1}^n \frac{1}{w^Tk(X)w}((\langle w,k(x,x_i)\rangle)_{i=1}^n)^T$.

In [127]:
def sample_given_q(x,w,q,kernel): #give kernel as a function in one argument
    
    sigma_12=torch.zeros_like(w)

    for i in range(len(x)):
        #returns sum_i w_i * kernel(x_i,x) a vector of len l
        sigma_12[i]= w @ kernel(x,x[i]) 

    
    sigma_11= w @ kernel(x,x) @ w.T #should be a scalar

    sigma_cond= kernel(x,x) - (torch.outer(sigma_12, sigma_12)*(1/sigma_11)) #outer product sigma_12*sigma_12.T
    mu_cond= sigma_12 *(q/sigma_11) #should be a vector of length l
    return mu_cond,sigma_cond

mu, cov= sample_given_q(x,w,5,gaussian_kernel)

def is_psd(mat, tol=1e-8):
    return bool((mat == mat.T).all() and (torch.linalg.eigvals(mat).real >= -tol).all())

#Test:
is_psd(cov)

print(sorted(torch.linalg.eigvals(cov).real))


[tensor(-2.0520e-15, dtype=torch.float64), tensor(-1.2771e-15, dtype=torch.float64), tensor(-1.2179e-15, dtype=torch.float64), tensor(-1.1038e-15, dtype=torch.float64), tensor(-1.0194e-15, dtype=torch.float64), tensor(-9.6093e-16, dtype=torch.float64), tensor(-9.4741e-16, dtype=torch.float64), tensor(-8.6751e-16, dtype=torch.float64), tensor(-8.2928e-16, dtype=torch.float64), tensor(-8.0853e-16, dtype=torch.float64), tensor(-8.0167e-16, dtype=torch.float64), tensor(-6.9700e-16, dtype=torch.float64), tensor(-6.6723e-16, dtype=torch.float64), tensor(-6.4902e-16, dtype=torch.float64), tensor(-6.2502e-16, dtype=torch.float64), tensor(-5.8566e-16, dtype=torch.float64), tensor(-5.8566e-16, dtype=torch.float64), tensor(-5.4814e-16, dtype=torch.float64), tensor(-4.9249e-16, dtype=torch.float64), tensor(-4.9249e-16, dtype=torch.float64), tensor(-4.3814e-16, dtype=torch.float64), tensor(-4.1322e-16, dtype=torch.float64), tensor(-4.1322e-16, dtype=torch.float64), tensor(-3.8640e-16, dtype=torch.f

In [131]:
#L = torch.linalg.cholesky(cov)
eigvals, eigvecs = torch.linalg.eigh(cov)
L = eigvecs @ torch.diag(torch.sqrt(eigvals.clamp(min=1e-8)))
L = torch.linalg.cholesky(cov)

_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 6 is not positive-definite).

In [123]:
mu,sigma=sample_given_q(x,w,10,gaussian_kernel)
print(mu, sigma)
dist = torch.distributions.MultivariateNormal(mu,sigma)

tensor([ 8.6687,  8.7417,  8.8137,  8.8849,  8.9551,  9.0243,  9.0925,  9.1596,
         9.2257,  9.2907,  9.3546,  9.4172,  9.4788,  9.5390,  9.5981,  9.6559,
         9.7124,  9.7675,  9.8213,  9.8737,  9.9248,  9.9744, 10.0225, 10.0692,
        10.1144, 10.1580, 10.2002, 10.2407, 10.2797, 10.3171, 10.3529, 10.3871,
        10.4196, 10.4504, 10.4796, 10.5071, 10.5328, 10.5569, 10.5792, 10.5997,
        10.6185, 10.6356, 10.6509, 10.6644, 10.6761, 10.6860, 10.6941, 10.7004,
        10.7049, 10.7077, 10.7086, 10.7077, 10.7049, 10.7004, 10.6941, 10.6860,
        10.6761, 10.6644, 10.6509, 10.6356, 10.6185, 10.5997, 10.5792, 10.5569,
        10.5328, 10.5071, 10.4796, 10.4504, 10.4196, 10.3871, 10.3529, 10.3171,
        10.2797, 10.2407, 10.2002, 10.1580, 10.1144, 10.0692, 10.0225,  9.9744,
         9.9248,  9.8737,  9.8213,  9.7675,  9.7124,  9.6559,  9.5981,  9.5390,
         9.4788,  9.4172,  9.3546,  9.2907,  9.2257,  9.1596,  9.0925,  9.0243,
         8.9551,  8.8849,  8.8137,  8.74

ValueError: Expected parameter covariance_matrix (Tensor of shape (101, 101)) of distribution MultivariateNormal(loc: torch.Size([101]), covariance_matrix: torch.Size([101, 101])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[ 0.3526,  0.3471,  0.3414,  ..., -0.2755, -0.2776, -0.2795],
        [ 0.3471,  0.3417,  0.3361,  ..., -0.2735, -0.2756, -0.2776],
        [ 0.3414,  0.3361,  0.3308,  ..., -0.2714, -0.2735, -0.2755],
        ...,
        [-0.2755, -0.2735, -0.2714,  ...,  0.3308,  0.3361,  0.3414],
        [-0.2776, -0.2756, -0.2735,  ...,  0.3361,  0.3417,  0.3471],
        [-0.2795, -0.2776, -0.2755,  ...,  0.3414,  0.3471,  0.3526]],
       dtype=torch.float64)